In [17]:
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
def undistort_video(input_path, output_path, calibration_images=None):
    """
    Remove lens distortion from a video file
    
    Parameters:
    - input_path: Path to the input video file
    - output_path: Path to save the undistorted video
    - calibration_images: Optional list of paths to chessboard images for calibration
                         If None, uses default estimation values
    """
    # Use estimated values for a typical wide-angle camera
    # These are default values, adjust based on your specific camera if known
    print("Using default camera parameters (no calibration images provided)")
    # Camera matrix (focal length and optical centers)
    mtx = np.array([
        [1000, 0, 960],  # fx, 0, cx
        [0, 1000, 540],  # 0, fy, cy
        [0, 0, 1]
    ])
    # Distortion coefficients [k1, k2, p1, p2, k3]
    dist = np.array([[-0.3, 0.1, 0, 0, -0.02]])

    # Open the video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Couldn't open video file {input_path}")
        return False
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Calculate optimal camera matrix
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (width, height), 1, (width, height))
    
    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    print(f"Processing video with {frame_count} frames...")
    frame_idx = 0
    
    # Process each frame
    with tqdm(total=frame_count) as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
                
            # Undistort the frame
            dst = cv2.undistort(frame, mtx, dist, None, newcameramtx)
            
            # Crop the image (optional)
            x, y, w, h = roi
            dst = dst[y:y+h, x:x+w]
            
            # Resize back to original dimensions if needed
            if dst.shape[1] != width or dst.shape[0] != height:
                dst = cv2.resize(dst, (width, height))
            
            # Write the undistorted frame
            out.write(dst)
            
            frame_idx += 1
            pbar.update(1)
    
    # Release resources
    cap.release()
    out.release()
    
    print(f"Undistorted video saved to {output_path}")
    return True

In [3]:
undistort_video("10secs.mp4", "undist.mp4")

Using default camera parameters (no calibration images provided)
Processing video with 301 frames...


100%|████████████████████████████████████████▊| 300/301 [00:14<00:00, 20.62it/s]

Undistorted video saved to undist.mp4


True

In [4]:
def detect_court_lines(frame):
    """
    Detect court lines using Hough Line Transform
    Returns the court corners if found
    """
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Use adaptive thresholding to identify court lines
    thresh = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 11, 2
    )
    
    # Find edges
    edges = cv2.Canny(thresh, 50, 150)
    
    # Dilate to connect edge components
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=1)
    
    # Find lines using HoughLinesP
    lines = cv2.HoughLinesP(
        dilated, 1, np.pi/180, threshold=50, 
        minLineLength=50, maxLineGap=10
    )
    
    if lines is None:
        return None
    
    # Draw lines on a copy of the frame for visualization
    line_frame = frame.copy()
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    # For debugging: save the line detection result
    cv2.imwrite("court_lines_detected.jpg", line_frame)
    
    # Group lines into horizontal and vertical
    h_lines = []
    v_lines = []
    
    for line in lines:
        x1, y1, x2, y2 = line[0]
        # Calculate angle
        angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
        
        # Group based on angle
        if angle < 45 or angle > 135:  # Horizontal lines
            h_lines.append(line[0])
        else:  # Vertical lines
            v_lines.append(line[0])
    
    # If we don't have enough lines, return None
    if len(h_lines) < 2 or len(v_lines) < 2:
        return None
    
    # Find boundary lines (furthest lines in each direction)
    h_lines = sorted(h_lines, key=lambda line: (line[1] + line[3]) / 2)  # Sort by y-coordinate
    v_lines = sorted(v_lines, key=lambda line: (line[0] + line[2]) / 2)  # Sort by x-coordinate
    
    top_line = h_lines[0]
    bottom_line = h_lines[-1]
    left_line = v_lines[0]
    right_line = v_lines[-1]
    
    # Find intersections of these lines to get court corners
    top_left = line_intersection(top_line, left_line)
    top_right = line_intersection(top_line, right_line)
    bottom_left = line_intersection(bottom_line, left_line)
    bottom_right = line_intersection(bottom_line, right_line)
    
    # Convert to integer points
    corners = [
        (int(top_left[0]), int(top_left[1])),
        (int(top_right[0]), int(top_right[1])),
        (int(bottom_right[0]), int(bottom_right[1])),
        (int(bottom_left[0]), int(bottom_left[1]))
    ]
    
    return corners

In [5]:
def line_intersection(line1, line2):
    """Find the intersection point of two lines"""
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2
    
    # Line 1 represented as a1x + b1y = c1
    a1 = y2 - y1
    b1 = x1 - x2
    c1 = a1 * x1 + b1 * y1
    
    # Line 2 represented as a2x + b2y = c2
    a2 = y4 - y3
    b2 = x3 - x4
    c2 = a2 * x3 + b2 * y3
    
    determinant = a1 * b2 - a2 * b1
    
    if determinant == 0:
        # Lines are parallel
        return (0, 0)
    else:
        x = (b2 * c1 - b1 * c2) / determinant
        y = (a1 * c2 - a2 * c1) / determinant
        return (x, y)


In [6]:
def manual_court_selection(frame):
    """
    Allow manual selection of court corners if automatic detection fails
    """
    corners = []
    
    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            corners.append((x, y))
            # Draw circle at selected point
            cv2.circle(display_frame, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow('Select Court Corners', display_frame)
            
            if len(corners) == 4:
                cv2.destroyWindow('Select Court Corners')
    
    # Create a copy of the frame for display
    display_frame = frame.copy()
    cv2.namedWindow('Select Court Corners')
    cv2.setMouseCallback('Select Court Corners', mouse_callback)
    
    # Display instructions
    text = "Click on 4 court corners in order: Top-Left, Top-Right, Bottom-Right, Bottom-Left"
    y_pos = 30
    for i, line in enumerate(text.split(', ')):
        cv2.putText(display_frame, line, (10, y_pos + i*30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.imshow('Select Court Corners', display_frame)
    cv2.waitKey(0)
    
    return corners if len(corners) == 4 else None

In [7]:
def apply_perspective_transform(frame, corners, target_width=800, target_height=400):
    """
    Apply perspective transformation to get a top-down view of the court
    """
    # Define the destination points (rectangle)
    dst_points = np.array([
        [0, 0],                      # Top-left
        [target_width, 0],           # Top-right
        [target_width, target_height], # Bottom-right
        [0, target_height]           # Bottom-left
    ], dtype=np.float32)
    
    # Convert corners to numpy array
    src_points = np.array(corners, dtype=np.float32)
    
    # Compute perspective transform matrix
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    
    # Apply transformation
    warped = cv2.warpPerspective(frame, M, (target_width, target_height))
    
    return warped, M

In [8]:
def process_video_with_perspective_transform(input_path, output_path):
    """
    Process a video to detect court and transform perspective
    """
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Couldn't open video file {input_path}")
        return False
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Read first frame for court detection
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read the first frame")
        return False
    
    # Try automatic court detection
    corners = detect_court_lines(first_frame)
    
    # If automatic detection fails, use manual selection
    if corners is None or len(corners) != 4:
        print("Automatic court detection failed. Please select court corners manually.")
        corners = manual_court_selection(first_frame)
        
        if corners is None or len(corners) != 4:
            print("Court corner selection failed")
            return False
    
    # Apply perspective transform
    target_width, target_height = 800, 400  # Standard padel court dimensions (scaled)
    _, perspective_matrix = apply_perspective_transform(first_frame, corners, target_width, target_height)
    
    # Create video writer for output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (target_width, target_height))
    
    # Reset video capture to beginning
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    print(f"Processing video with {frame_count} frames...")
    
    # Process each frame
    with tqdm(total=frame_count) as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Apply perspective transform
            warped_frame, _ = apply_perspective_transform(
                frame, corners, target_width, target_height
            )
            
            # Write transformed frame
            out.write(warped_frame)
            
            pbar.update(1)
    
    # Release resources
    cap.release()
    out.release()
    
    print(f"Perspective-corrected video saved to {output_path}")
    
    # Save the perspective matrix for future use in player tracking
    np.save("perspective_matrix.npy", perspective_matrix)
    print("Perspective transformation matrix saved to perspective_matrix.npy")
    
    return True, perspective_matrix

In [9]:
succ, trans = process_video_with_perspective_transform("undist.mp4", "new.mp4")

Processing video with 300 frames...


100%|████████████████████████████████████████| 300/300 [00:02<00:00, 133.71it/s]

Perspective-corrected video saved to new.mp4
Perspective transformation matrix saved to perspective_matrix.npy


In [10]:
def enhance_court_features(frame):
    """
    Enhance the visibility of court lines and features
    """
    # Convert to HSV for better color segmentation
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Create mask for white lines
    # White in HSV has high V, low S, and any H
    lower_white = np.array([0, 0, 180])
    upper_white = np.array([180, 70, 255])
    white_mask = cv2.inRange(hsv, lower_white, upper_white)
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Apply contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    
    # Combine white mask with enhanced image
    combined = cv2.bitwise_or(enhanced, enhanced, mask=white_mask)
    
    # Save intermediate results for debugging
    cv2.imwrite("white_mask.jpg", white_mask)
    cv2.imwrite("enhanced_features.jpg", combined)
    
    return combined, white_mask

In [11]:
def detect_specific_court_lines(frame):
    """
    Detect specific court lines: bottom field line, middle line, and net line
    """
    height, width = frame.shape[:2]
    
    # Enhance court features
    enhanced, white_mask = enhance_court_features(frame)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(enhanced, (5, 5), 0)
    
    # Edge detection
    edges = cv2.Canny(blurred, 50, 150)
    
    # Dilate to connect edge components
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(edges, kernel, iterations=1)
    
    # Save intermediate results for debugging
    cv2.imwrite("edges_detected.jpg", edges)
    
    # Detect lines using Hough Line Transform
    lines = cv2.HoughLinesP(
        dilated, 1, np.pi/180, threshold=50, 
        minLineLength=width//3,  # At least 1/3 of width to filter small noisy lines
        maxLineGap=20
    )
    
    if lines is None or len(lines) < 3:
        print("Not enough lines detected, try using manual selection")
        return None
    
    # Visualization for debugging
    line_frame = frame.copy()
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    cv2.imwrite("all_lines_detected.jpg", line_frame)
    
    # Separate horizontal and vertical lines
    h_lines = []
    v_lines = []
    
    for line in lines:
        x1, y1, x2, y2 = line[0]
        # Calculate angle and length
        angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)
        length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        
        # Group based on angle
        if angle < 30 or angle > 150:  # Horizontal-ish lines
            h_lines.append((line[0], y1, length))  # Store with y-coordinate and length
        elif 60 < angle < 120:  # Vertical-ish lines
            v_lines.append((line[0], x1, length))  # Store with x-coordinate and length
    
    # Sort horizontal lines by y-coordinate (top to bottom)
    h_lines.sort(key=lambda x: x[1])
    
    # Get specific horizontal lines we need
    if len(h_lines) < 2:
        print("Not enough horizontal lines detected")
        return None
    
    # Find the net line (typically in the upper half, and usually one of the stronger white lines)
    # Use white mask to help identify it
    net_line_candidates = []
    
    for line_data in h_lines:
        line, y_coord, length = line_data
        x1, y1, x2, y2 = line
        
        # Check if this line is in the upper half of the image
        if y1 < height // 2:
            # Create a thin mask around this line
            line_mask = np.zeros_like(white_mask)
            cv2.line(line_mask, (x1, y1), (x2, y2), 255, 5)
            
            # Count white pixels along this line in the white mask
            white_pixels = cv2.countNonZero(cv2.bitwise_and(white_mask, line_mask))
            
            # Add to candidates with a score (white pixels * length)
            score = white_pixels * length
            net_line_candidates.append((line_data, score))
    
    # Sort by score and select top candidate
    if net_line_candidates:
        net_line_candidates.sort(key=lambda x: x[1], reverse=True)
        net_line_data = net_line_candidates[0][0]
    else:
        # Fallback: use the highest line in upper half
        upper_lines = [l for l in h_lines if l[1] < height // 2]
        if upper_lines:
            net_line_data = upper_lines[0]
        else:
            print("Could not identify net line")
            return None
    
    # Find bottom field line (should be in lower part of image)
    bottom_line_candidates = [l for l in h_lines if l[1] > height * 0.6]
    
    if bottom_line_candidates:
        # Sort by length to get the most complete line
        bottom_line_candidates.sort(key=lambda x: x[2], reverse=True)
        bottom_line_data = bottom_line_candidates[0]
    else:
        # Fallback: use the lowest horizontal line
        bottom_line_data = h_lines[-1]
    
    # Find middle court line (between net and bottom)
    mid_y = (net_line_data[1] + bottom_line_data[1]) / 2
    middle_line_data = min(h_lines, key=lambda x: abs(x[1] - mid_y))
    
    # Get the actual line data
    net_line = net_line_data[0]
    middle_line = middle_line_data[0]
    bottom_line = bottom_line_data[0]
    
    # Now find vertical lines (usually the sides of the court)
    if len(v_lines) < 2:
        print("Not enough vertical lines detected")
        return None
    
    # Sort by x-coordinate
    v_lines.sort(key=lambda x: x[1])
    
    # Get leftmost and rightmost vertical lines
    left_line = v_lines[0][0]
    right_line = v_lines[-1][0]
    
    # Visualization for debugging - show the specific lines we're using
    key_lines_frame = frame.copy()
    cv2.line(key_lines_frame, (net_line[0], net_line[1]), (net_line[2], net_line[3]), (255, 0, 0), 3)  # Net line in blue
    cv2.line(key_lines_frame, (middle_line[0], middle_line[1]), (middle_line[2], middle_line[3]), (0, 255, 0), 3)  # Middle line in green
    cv2.line(key_lines_frame, (bottom_line[0], bottom_line[1]), (bottom_line[2], bottom_line[3]), (0, 0, 255), 3)  # Bottom line in red
    cv2.line(key_lines_frame, (left_line[0], left_line[1]), (left_line[2], left_line[3]), (255, 255, 0), 3)  # Left line in cyan
    cv2.line(key_lines_frame, (right_line[0], right_line[1]), (right_line[2], right_line[3]), (255, 0, 255), 3)  # Right line in magenta
    
    cv2.imwrite("key_court_lines.jpg", key_lines_frame)
    
    # Calculate intersections to find the court corners
    try:
        # Top corners (using net line)
        top_left = line_intersection(net_line, left_line)
        top_right = line_intersection(net_line, right_line)
        
        # Bottom corners
        bottom_left = line_intersection(bottom_line, left_line)
        bottom_right = line_intersection(bottom_line, right_line)
        
        # Convert to integer points
        corners = [
            (int(top_left[0]), int(top_left[1])),
            (int(top_right[0]), int(top_right[1])),
            (int(bottom_right[0]), int(bottom_right[1])),
            (int(bottom_left[0]), int(bottom_left[1]))
        ]
        
        # Draw the detected corners
        corner_frame = frame.copy()
        for i, corner in enumerate(corners):
            cv2.circle(corner_frame, corner, 10, (0, 0, 255), -1)
            cv2.putText(corner_frame, f"Corner {i+1}", (corner[0]+10, corner[1]), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        
        cv2.imwrite("detected_corners.jpg", corner_frame)
        
        return corners
    
    except Exception as e:
        print(f"Error calculating intersections: {e}")
        return None

In [12]:
def line_intersection(line1, line2):
    """Find the intersection point of two lines"""
    x1, y1, x2, y2 = line1
    x3, y3, x4, y4 = line2
    
    # Line 1 represented as a1x + b1y = c1
    a1 = y2 - y1
    b1 = x1 - x2
    c1 = a1 * x1 + b1 * y1
    
    # Line 2 represented as a2x + b2y = c2
    a2 = y4 - y3
    b2 = x3 - x4
    c2 = a2 * x3 + b2 * y3
    
    determinant = a1 * b2 - a2 * b1
    
    if determinant == 0:
        # Lines are parallel
        raise Exception("Lines are parallel, no intersection")
    else:
        x = (b2 * c1 - b1 * c2) / determinant
        y = (a1 * c2 - a2 * c1) / determinant
        return (x, y)

In [13]:
def manual_court_selection_with_guidance(frame):
    """
    Allow manual selection of court corners with guidance images
    """
    corners = []
    
    def mouse_callback(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            corners.append((x, y))
            # Draw circle at selected point
            cv2.circle(display_frame, (x, y), 10, (0, 0, 255), -1)
            
            # Label the point
            point_label = ["Top-Left", "Top-Right", "Bottom-Right", "Bottom-Left"][len(corners) - 1]
            cv2.putText(display_frame, point_label, (x+10, y), 
                      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            cv2.imshow('Select Court Corners', display_frame)
            
            if len(corners) == 4:
                cv2.waitKey(1000)  # Wait a bit to show the final point
                cv2.destroyWindow('Select Court Corners')
    
    # Create a copy of the frame for display with guidance overlay
    display_frame = frame.copy()
    
    # Add guidance overlay - court diagram
    h, w = frame.shape[:2]
    overlay_margin = int(h * 0.1)
    overlay_size = (int(w * 0.25), int(h * 0.25))
    
    # Create a simple court diagram
    court_diagram = np.ones((overlay_size[1], overlay_size[0], 3), dtype=np.uint8) * 255
    cv2.rectangle(court_diagram, (5, 5), (overlay_size[0]-5, overlay_size[1]-5), (0, 0, 0), 2)
    cv2.line(court_diagram, (5, overlay_size[1]//2), (overlay_size[0]-5, overlay_size[1]//2), (0, 0, 0), 2)
    
    # Add corner labels
    cv2.putText(court_diagram, "1", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(court_diagram, "2", (overlay_size[0]-20, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(court_diagram, "3", (overlay_size[0]-20, overlay_size[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.putText(court_diagram, "4", (10, overlay_size[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
    # Place the diagram in the corner of the display frame
    display_frame[overlay_margin:overlay_margin+overlay_size[1], 
                 overlay_margin:overlay_margin+overlay_size[0]] = court_diagram
    
    # Display instructions
    instruction_text = [
        "Select 4 court corners in this order:",
        "1: Top-Left (net side)",
        "2: Top-Right (net side)", 
        "3: Bottom-Right (back of court)",
        "4: Bottom-Left (back of court)"
    ]
    
    y_pos = overlay_margin + overlay_size[1] + 30
    for i, line in enumerate(instruction_text):
        cv2.putText(display_frame, line, (overlay_margin, y_pos + i*30), 
                  cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    cv2.namedWindow('Select Court Corners')
    cv2.setMouseCallback('Select Court Corners', mouse_callback)
    cv2.imshow('Select Court Corners', display_frame)
    cv2.waitKey(0)
    
    return corners if len(corners) == 4 else None

In [14]:
def apply_perspective_transform(frame, corners, target_width=800, target_height=400):
    """
    Apply perspective transformation to get a top-down view of the court
    """
    # Define the destination points (rectangle)
    dst_points = np.array([
        [0, 0],                       # Top-left
        [target_width, 0],            # Top-right
        [target_width, target_height],  # Bottom-right
        [0, target_height]            # Bottom-left
    ], dtype=np.float32)
    
    # Convert corners to numpy array
    src_points = np.array(corners, dtype=np.float32)
    
    # Compute perspective transform matrix
    M = cv2.getPerspectiveTransform(src_points, dst_points)
    
    # Apply transformation
    warped = cv2.warpPerspective(frame, M, (target_width, target_height))
    
    return warped, M

In [15]:
def process_video_with_improved_perspective(input_path, output_path):
    """
    Process a video to detect court using improved methods and transform perspective
    """
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Couldn't open video file {input_path}")
        return False
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Read first frame for court detection
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to read the first frame")
        return False
    
    # Try automatic court detection with improved method
    print("Attempting automatic court detection...")
    corners = detect_specific_court_lines(first_frame)
    
    # If automatic detection fails, use guided manual selection
    if corners is None or len(corners) != 4:
        print("Automatic court detection failed. Please select court corners manually.")
        corners = manual_court_selection_with_guidance(first_frame)
        
        if corners is None or len(corners) != 4:
            print("Court corner selection failed")
            return False
    
    # Apply perspective transform
    target_width, target_height = 800, 400  # Standard padel court dimensions (scaled)
    warped_first_frame, perspective_matrix = apply_perspective_transform(
        first_frame, corners, target_width, target_height
    )
    
    # Save the first transformed frame for verification
    cv2.imwrite("first_frame_transformed.jpg", warped_first_frame)
    
    # Create video writer for output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (target_width, target_height))
    
    # Reset video capture to beginning
    cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    print(f"Processing video with {frame_count} frames...")
    
    # Process each frame
    with tqdm(total=frame_count) as pbar:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            # Apply perspective transform
            warped_frame, _ = apply_perspective_transform(
                frame, corners, target_width, target_height
            )
            
            # Write transformed frame
            out.write(warped_frame)
            
            pbar.update(1)
    
    # Release resources
    cap.release()
    out.release()
    
    print(f"Perspective-corrected video saved to {output_path}")
    
    # Save the perspective matrix for future use in player tracking
    np.save("perspective_matrix.npy", perspective_matrix)
    print("Perspective transformation matrix saved to perspective_matrix.npy")
    
    return True, perspective_matrix

In [16]:
success, transform_matrix = process_video_with_improved_perspective("10secs.mp4", "v2.mp4")

Attempting automatic court detection...
Not enough lines detected, try using manual selection
Automatic court detection failed. Please select court corners manually.
Processing video with 301 frames...


100%|███████████████████████████████████████▊| 300/301 [00:02<00:00, 127.17it/s]

Perspective-corrected video saved to v2.mp4
Perspective transformation matrix saved to perspective_matrix.npy
